In [ ]:
import tensorflow as tf
from tensorflow.keras import models , layers
import matplotlib.pyplot as plt

## Download Images into tf.images.dataset

In [ ]:
Image_size = 256
Batch_size = 32
Channels = 3

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'Potato_Data',
    shuffle=True,
    image_size= (Image_size,Image_size),
    batch_size= Batch_size
)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
len(dataset)

## Data visualization

In [ ]:
for image_batch , label_batch in dataset.take(1) :  # It gives us one batch which equals to 32 images 
    print(image_batch[0].numpy()) # because it is a tensor 


In [ ]:


plt.figure(figsize=(10,8))
for image_batch , label_batch in dataset.take (1) :
    for i in range(12) :
        ax = plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        plt.axis('off')
        plt.title(class_names[label_batch[i]])

## Train Test Split

In [ ]:
len(dataset)

In [ ]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset,len(dataset))

In [ ]:
print('Length of Train ds :',len(train_ds))
print('Length of Validation ds :',len(val_ds))
print('Length of Test ds :',len(test_ds))

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resizing_rescaling = tf.keras.Sequential([
    
    layers.experimental.preprocessing.Resizing(Image_size,Image_size),
    layers.experimental.preprocessing.Rescaling(1.0/255)
    
])

In [ ]:
data_augmentation = tf.keras.Sequential([
    
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2)
    
])

# Model building

In [ ]:
input_shape = (Batch_size,Image_size,Image_size,Channels)
n_classes = 3 
model = models.Sequential([
    
    resizing_rescaling,
    data_augmentation,
    layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax')
])

model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile( 
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics='accuracy'
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=50,
    batch_size=Batch_size,
    verbose=1,
    validation_data=val_ds,
    
)